In [1]:
!pip3 install openai
!pip3 install --upgrade openai

Defaulting to user installation because normal site-packages is not writeable
--- Logging error ---
Traceback (most recent call last):
  File "/Users/joshua.grefte/Library/Python/3.10/lib/python/site-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/Users/joshua.grefte/Library/Python/3.10/lib/python/site-packages/pip/_vendor/rich/console.py", line 1752, in print
    extend(render(renderable, render_options))
  File "/Users/joshua.grefte/Library/Python/3.10/lib/python/site-packages/pip/_vendor/rich/console.py", line 1390, in render
    for render_output in iter_render:
  File "/Users/joshua.grefte/Library/Python/3.10/lib/python/site-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/Users/joshua.grefte/Library/Python/3.10/lib/python/site-packages/pip/_vendor/rich/segment.py", line 245, in split_lines
    for segment in segments:
  File "/

In [2]:
#https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
#https://www.youtube.com/watch?v=uCKH8bmPgFs&t=502s

In [3]:
#create resource group and resource and deploy gpt4 model then use api key and endpoint value

In [22]:
import openai
openai.api_type = "azure"
openai.api_version = "2023-05-15" 
openai.api_base = "https://gpt4resource.openai.azure.com/" # Your Azure OpenAI resource's endpoint value.
openai.api_key = "618bf90290544e54b47bcfc2dba743da"

system_message = 'Assistant is an intelligent chatbot designed to help users create programming training materials'
user_message1 = 'Could you please provide me with incorrect code in Node.js, followed by a corresponding unit test that will fail due to the introduced bug? At the end provide the corrected code. Area of interest is web development.'
response = openai.ChatCompletion.create(
    #gpt-35-turbo max tokens = 4,096
    engine="test-gpt-35-turbo", # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message1}
    ],
    temperature = 0 # This is the degree of randomness of the model's output
)

print(response)
print(50*'-')
output = response['choices'][0]['message']['content']
print('output: ', output)
print(50*'-')
print('tokens: ', response['usage'])

RateLimitError: The server is currently overloaded with other requests. Sorry about that! You can retry your request, or contact us through an Azure support request at: https://go.microsoft.com/fwlink/?linkid=2213926 if the error persists.

In [28]:
#RUN CELL TO SAVE OUTPUT IN A TXT FILE
import os

folder_path = 'txt_outputs'
text_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]
txt_number_list = []
for file_name in text_files:
    txt_number = file_name[-5]
    txt_number = int(txt_number)
    txt_number_list.append(txt_number)
if not txt_number_list: #check if list is empty, i.e. no txt files yet
    new_txt_number = str(1)
else:
    new_txt_number = str(max(txt_number_list) + 1)
text_file = open('txt_outputs/output' + '_' + new_txt_number + '.txt', 'w')
text_file.write(system_message + '\n')
text_file.write(50*'-' + '\n')
text_file.write(user_message1 + '\n')
text_file.write(50*'-' + '\n')
text_file.write(output + '\n')
text_file.close()